# 微分方程建模实例

## 1.Malthus模型

### 1.模型假设
1. 设$x(t)$表示$t$时刻的人口数，且$x(t)$连续可微
2. 人口的增长率$r$是常数(增长率=出生率-死亡率)
3. 人口数量的变化是封闭的，即人口数量的增加与减少只取决于人口中个体的生于和死亡，且每个个体都具有同样的生育能力和死亡率
### 2.建模与求解

由假设，$t$时刻到$t+\Delta t$时刻人口的增量为$x(t+\Delta t)-x(t) = rx(t)\Delta t$，于是$\frac{dx}{dt} = rx$;  $x(0) = x0$

In [8]:
from sympy import *

x = Function('x')
t, r, x0 = symbols('t r x0')
eq = diff(x(t),t)-r*x(t)
xt = dsolve(eq,x(t), ics={x(0):x0})
print(xt)

Eq(x(t), x0*exp(r*t))


## 2.Logistic模型

对上面的增长率$r$进行修正：

1. 如果在人口较少时，可以把增长率看成常数
2. 当人口增加到一定数量之后，就应当视$r$为一个随着人口的增加而减少的量，即将增长率$r$表示为人口$x(t)$的函数$r(x)$，且$r(x)$为$x$的减函数

### 1.模型假设
1. 设$r(x)$为$x$的线性函数，$r(x) = r - sx$(工程师原则，首先用线性)
2. 自然资源与环境条件所能容纳的最大人口数为$x_m$，即当$x = x_m$时，增长率$r(x_m) = 0$
### 2.建模与求解
$r(x) = r(1-\frac{x}{x_m})$，则有$\frac{dx}{dt} = r(1-\frac{x}{x_m})$;  $x(t_0) = x0$

In [13]:
import sympy as sp

t, r, xm, t0, x0 = sp.var('t r xm t0 x0')
x = sp.Function('x')
eq = sp.diff(x(t),t) - r*(1-x(t)/xm)*x(t)
xt = sp.dsolve(eq, x(t), ics={x(t0):x0})
print(xt)

Eq(x(t), xm/(1 - (x0 - xm)*exp(-r*t + r*t0)/x0))


1. 非线性最小二乘法

In [12]:
import numpy as np
from scipy.optimize import curve_fit

a = []; b = []
with open('Pdata8_10_1.txt') as f:
    s = f.read().splitlines()  #返回每一列的数据
# print(s)
for i in range(0, len(s),2):  #读入奇数行的数据
    d1 = s[i].split('\t')
    for j in range(len(d1)):
        if d1[j] != '':
            a.append(eval(d1[j]))  #把非空的字符串转换为年代数据
for i in range(1,len(s),2):  #读入偶数行数据
    d2 = s[i].split('\t')
    for j in range(len(d2)):
        if d2[j] != '':
            b.append(eval(d2[j]))

c = np.vstack((a,b))
# print(c)
np.savetxt('Year_human_data.txt', c)  #保存数据

x = lambda t,r,xm: xm/(1+(xm/3.9-1)*np.exp(-r*(t-1790)))
bd = ((0,200),(0.1,1000))  #约束两个参数的下界和上界
popt, pcov = curve_fit(x, a[1:], b[1:], bounds=bd)
print(popt)
print('2010年的预测值为:', x(2010, *popt))
print('参数的协方差矩阵\n', pcov)

[2.73527906e-02 3.42441913e+02]
2010年的预测值为: 282.67978321958697
参数的协方差矩阵
 [[ 1.75174192e-07 -5.68466764e-03]
 [-5.68466764e-03  2.38473411e+02]]


2. 线性最小二乘法

为了利用简单的线性最小二乘估计这个模型的参数$r$和$x_m$，把Logistic方程表示为

$\frac{1}{x}\frac{dx}{dt} = r-sx$， $s = \frac{r}{xm}$

记1790，1800，……，2000年分别用$k = 1,2,···,22$表示，利用向前差分，得到差分方程

$\frac{1}{x(k)·\frac{x(k+1)-x(k)}{\Delta t} = r-sx(k),k = 1,2,···,21}$

其中步长$\Delta t = 10$

In [16]:
import numpy as np

d = np.loadtxt('Year_human_data.txt')
t0 = d[0]; x0 = d[1]
# print(x0)
a = np.vstack([np.ones(len(x0)-1), -x0[:-1]]).T  #构造线性方程组系数矩阵
b = np.diff(x0)/10/x0[:-1]  #构造线性方程组的常数项列

rs = np.linalg.pinv(a)@b
r = rs[0]; xm = r/rs[1]
print('人口增长率r和人口最大值xm的拟合值分别为:', np.round([r,xm], 4))

xhat = xm/(1+(xm/3.9-1)*np.exp(-r*(2010-1790)))  #求预测值
print('2010年的预测值为:', round(xhat,4))

人口增长率r和人口最大值xm的拟合值分别为: [3.25000e-02 2.94386e+02]
2010年的预测值为: 277.9634
